# Критерии syntactic complexity

### 1. Количество слов

In [2]:
from model import Model
import re
import copy
import numpy as np

#### Функция, которая добавляет пробелы после .?!

In [3]:
def space(string):
    string = re.sub('([a-zA-Z]| )([\.\?!])', '\\1\\2 ', string)
    string = re.sub(': ', ' : ', string)
    string = re.sub('; ', ' ; ', string)
    string = re.sub('  +', ' ', string)
    return string

#### Функция, которая парсит текст udpipe. На выходе получается строка с форматом 'conllu'.

In [4]:
def get_parsed_text(model_name, text_path):
    model = Model(model_name)
    text = open(text_path, 'r')
    text = text.read()
    text = space(text)
    sentences = model.tokenize(text)
    for s in sentences:
        model.tag(s)
        model.parse(s)
    output = model.write(sentences, "conllu")
    return output

#### Функция, которая считает токены в тексте. Не считаются знаки препинания.

In [5]:
def count_tokens(parsed_text):
    num_tokens = 0
    lst_str = parsed_text.split('\n')
    for every_str in lst_str:
        #print(every_str)
        if ('PUNCT' not in every_str) and every_str.startswith('#') == False:
            #print(every_str)
            num_tokens += 1
    return num_tokens

### 2. Глубина дерева

#### Функция, которая считает глубину дерева.

In [6]:
def order_head(parsed_sent):
    sent_lst = parsed_sent.split('\n')
    #print(sent_lst[0])
    order_head_lst = []
    for token in sent_lst:
        token = re.sub(r'\|', '$', token)
        if '\t_\t_\t_\t_\t_' not in token:
            order = re.search('([0-9]+)\t', token).group(1)
        #print(order)
            head = re.search('.+\t.+\t([0-9]+)', token).group(1)
        #print(head)
            token = re.search('^[0-9]+\t(.+?)\t', token).group(1)
            order_head_lst.append((int(order), int(head), token))
    return order_head_lst

In [7]:
def find_root(order_head_lst):
    for every_order_head in order_head_lst:
        if every_order_head[1] == 0:
            root = every_order_head
    return root

In [8]:
def root_children(parsed_sent):
    order_head_lst = order_head(parsed_sent)
    #print(order_head_lst)
    root = find_root(order_head_lst)
    chains = []
    for every_order_head in order_head_lst:
        if every_order_head[1] == root[0]:
            chains.append([root[0], every_order_head[0]])
    return chains, order_head_lst

In [9]:
def chains_heads(chains, order_head_lst):
    length_chains = len(chains)
    i = 0
    for chain in chains:
        if i < length_chains:
            heads = []
            if 'stop' not in chain:
                for order_head in order_head_lst:
                    if chain[-1] == order_head[1]:
                        heads.append(order_head[0])
                if heads == [] and 'stop' not in chain:
                    chain.append('stop')
                else:
                    ind_head = 0
                    for head in heads:
                        new_chain = copy.copy(chain)[:-1]
                        if ind_head == 0:
                            chain.append(head)
                            ind_head += 1
                        else:
                            new_chain.append(head)
                            chains.append(new_chain)
        i += 1
    while all(item[-1] == 'stop' for item in chains) is False:
        chains_heads(chains, order_head_lst)
    return chains

In [10]:
def count_max_depth_for_one_sent(sent):
    chains, order_head_lst = root_children(sent)
    chains = chains_heads(chains, order_head_lst)
    depths = []
    #print(chains)
    for chain in chains:
        depths.append(len(chain)-2)
    return max(depths)

In [11]:
def count_depths_for_one_text(parsed_text):
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    max_depths = []
    for sent in sent_lst:
        #print(sent)
        max_depths.append(count_max_depth_for_one_sent(sent))
    return max_depths

In [12]:
def av_depth_for_one_text(parsed_text):
    max_depths = count_depths_for_one_text(parsed_text)
    return np.mean(max_depths)

In [13]:
def max_depth_for_one_text(parsed_text):
    max_depths = count_depths_for_one_text(parsed_text)
    return np.max(max_depths)

In [14]:
def min_depth_for_one_text(parsed_text):
    max_depths = count_depths_for_one_text(parsed_text)
    return np.min(max_depths)

In [15]:
#count_depths_for_one_text(parsed_text)

In [16]:
#av_depth_for_one_text(parsed_text)

In [17]:
#max_depth_for_one_text(parsed_text)

In [18]:
#min_depth_for_one_text(parsed_text)

### 3. Количество зависимых клауз: acl, acl:relcl, advcl

In [19]:
# Возвращает словарь, где ключ - номер предложения, значения - массив [кол-во acl, кол-во acl:relcl, кол-во advcl]
def count_dependent_sent(parsed_text):
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    num_sent = 1
    d_sent = {}
    for sent in sent_lst:
        acl = len(re.findall('\t(acl)\t',sent))
        relcl = len(re.findall('\t(acl:relcl)\t',sent))
        advcl = len(re.findall('\t(advcl)\t',sent))
        d_sent[num_sent] = [acl, relcl, advcl]
        num_sent += 1
    return d_sent

In [20]:
def count_dependent_sent_text(parsed_text):
    d_sent = count_dependent_sent(parsed_text)
    acl = 0
    rel_cl = 0
    advcl = 0
    for sent in d_sent:
        acl = acl + d_sent[sent][0]
        rel_cl = rel_cl + d_sent[sent][1]
        advcl = advcl + d_sent[sent][2]
    return acl, rel_cl, advcl

In [21]:
#count_dependent_sent(parsed_text)

In [22]:
#count_dependent_sent_text(parsed_text)

### 4. Количество предложений

In [23]:
def count_sent(parsed_text):
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    return len(sent_lst)

In [24]:
#count_sent(parsed_text)

### 5. Количество клауз

In [25]:
def parsing_things(string):
    token = re.search('[0-9]+\t(.+?)\t', string).group(1)
    order = re.search('([0-9]+)\t', string).group(1)
    head = re.search('\t([0-9]+)\t', string).group(1)
    rel_type = re.search('\t[0-9]+\t(.+?)\t', string).group(1)
    pos = re.search('[0-9]+\t.+?\t.+?\t(.+?)\t', string).group(1)
    #grammar = re.search('[VERB|AUX]\t.+?\t(.+?)\t', every_str).group(1)
    return order, token, head, rel_type, pos

#### Функция, которая возвращает словарь, где ключи - номера предложений, а значения - количество клауз.

***There was a woman next door, and she was a singer.*** - 2 T-units, 2 clauses

***There was a woman next door who was a singer.*** - 1 T-units, 2 clauses

***But while they were trying they killed a whale and used the oil for the lamps.*** - 2 clauses

In [26]:
def count_clauses_every_sent(parsed_text):
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    verb_cl = {}
    all_num_sent = count_sent(parsed_text)
    for sent in range(1, all_num_sent+1):
        verb_cl[sent] = []
    num_sent = 1
    for every_sent in sent_lst:
        lst_str = every_sent.split('\n')
        for every_str in lst_str:
            if ('VerbForm=Fin' in every_str):
                sent_id = str(num_sent)
                order, token, head, rel_type, pos = parsing_things(every_str)
                if head not in verb_cl[int(sent_id)] and rel_type != 'conj':
                    verb_cl[int(sent_id)].append([order, head])
        num_sent += 1
    for key, value in verb_cl.items():
        if verb_cl[key] == []:
            verb_cl[key] = [None]
        verb_cl[key] = len(verb_cl[key])
    return verb_cl

In [27]:
def count_clauses(parsed_text):
    verb_cl = count_clauses_every_sent(parsed_text)
    num_cl = 0
    for key, value in verb_cl.items():
        num_cl = num_cl + value
    return num_cl

In [28]:
#count_clauses(parsed_text)

### 6. Количество T-юнитов

In [29]:
def find_subjects(sentence):
    lst_str = sentence.split('\n')
    maybe_depends = []
    for every_str in lst_str:
        finding = re.search('PRON|NOUN', every_str)
        if '_\t_\t_\t_\t_' not in every_str:
            order, token, head, rel_type, pos = parsing_things(every_str)
            if finding is not None:
                maybe_depends.append(rel_type)
    return maybe_depends

In [30]:
def count_tunits_every_sent(parsed_text):
    verb_cl = count_clauses_every_sent(parsed_text)
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    for key, value in verb_cl.items():
        subjects = find_subjects(sent_lst[key-1])
        acl_relcl = subjects.count('acl:relcl')
        acl = subjects.count('acl')
        advcl = subjects.count('advcl')
        depends = acl_relcl + acl + advcl
        verb_cl[key] = verb_cl[key]-depends
    return verb_cl

In [31]:
def count_tunits(parsed_text):
    verb_cl = count_tunits_every_sent(parsed_text)
    num_t = 0
    for key, value in verb_cl.items():
        num_t = num_t + value
    return num_t

In [32]:
#parsed_text = get_parsed_text('english-partut-ud-2.0-170801.udpipe', '/Users/irene/Desktop/Курсовая/esseys/AAl_13_2.txt')

In [33]:
#(parsed_text)

### 7. Количество сложных T-юнитов

In [34]:
def count_complex_tunit(parsed_text):
    return count_clauses(parsed_text)-count_tunits(parsed_text)

In [35]:
#count_complex_tunit(parsed_text)

### 8. Количество сочинительных фраз

#### Функция возвращает все сочинительные союзы (их вершины): ключи - номера предложений, значения - массив из вершин союзов.

In [43]:
def find_all_coord(parsed_text):
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    all_num_sent = count_sent(parsed_text)
    cp = {}
    for sent in range(1, all_num_sent+1):
        cp[sent] = []
    num_sent = 1
    for every_sent in sent_lst:
        lst_str = every_sent.split('\n')
        for every_str in lst_str:
            if '\tcc\t' in every_str:
                order, token, head, rel_type, pos = parsing_things(every_str)
                cp[int(num_sent)].append(head)
        num_sent += 1
    return cp

#### Функция возвращает cловарь: ключи - номера предложений, значение - количество сочинительных фраз.

In [44]:
def final_coord(parsed_text):
    cp = find_all_coord(parsed_text)
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    all_num_sent = count_sent(parsed_text)
    num_sent = 1
    cp_final = {}
    for sent in range(1, all_num_sent+1):
        cp_final[sent] = 0
    for every_sent in sent_lst:
        #print('НОМЕР ПРЕДЛОЖЕНИЯ: ' + str(num_sent))
        for every_coord_cp in cp[num_sent]:
            #print('ВЕРШИНА СОЮЗА: ' + every_coord_cp)
            finding2 = re.search('('+every_coord_cp+'\t.+?\tconj.+?)\n', every_sent)
            if finding2 is None:
                continue
            else:
                order2, token2, head2, rel_type2, pos2 = parsing_things(finding2.group(0))
                #print('2 СВЯЗУЮЩИЙ: ' + token2 + pos2)
                finding1 = re.search('('+head2+'\t.+?\t[a-z]+.+?)\n', every_sent)
                order1, token1, head1, rel_type1, pos1 = parsing_things(finding1.group(0))
                #print('1 СВЯЗУЮЩИЙ: ' + token1+pos1)
                if pos2 == pos1:
                    cp_final[num_sent] += 1 
        num_sent += 1
    return cp_final

In [45]:
def count_coord(parsed_text):
    cp_final = final_coord(parsed_text)
    num_cp = 0
    for key, value in cp_final.items():
        num_cp = num_cp + value
    return num_cp

In [46]:
#parsed_text = get_parsed_text('english-partut-ud-2.0-170801.udpipe', '/Users/irene/Desktop/Курсовая/esseys/AAl_27_1.txt')

In [47]:
#count_coord(parsed_text)

5

### 9. Количество сложных именных групп

Complex nominals comprise (i) nouns plus adjective, possessive, prepositional phrase, relative clause, participle, or appositive, (ii) nominal clauses, and (iii) gerunds and infinitives in subject position (Cooper 1976)

#### Possessive

In [ ]:
def find_possesive(parsed_text):
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    all_num_sent = count_sent(parsed_text)
    poss = {}
    for sent in range(1, all_num_sent+1):
        poss[sent] = 0
    num_sent = 1
    for every_sent in sent_lst:
        lst_str = every_sent.split('\n')
        for every_str in lst_str:
            if 'nmod' in every_str:# включая nmod:poss
                order, token, head, rel_type, pos = parsing_things(every_str)
                poss[num_sent] = poss[num_sent] + 1
        num_sent += 1
    return poss

#### Prepositional phrase

In [ ]:
def find_prep(parsed_text):
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    all_num_sent = count_sent(parsed_text)
    poss = {}
    for sent in range(1, all_num_sent+1):
        poss[sent] = 0
    num_sent = 1
    for every_sent in sent_lst:
        lst_str = every_sent.split('\n')
        for every_str in lst_str:
            if 'ADP' in every_str:
                order, token, head, rel_type, pos = parsing_things(every_str)
                poss[num_sent] = poss[num_sent] + 1
        num_sent += 1
    return poss

#### Nouns plus adjective

In [ ]:
def find_nouns(parsed_text):
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    all_num_sent = count_sent(parsed_text)
    vp = {}
    for sent in range(1, all_num_sent+1):
        vp[sent] = []
    num_sent = 1
    for every_sent in sent_lst:
        lst_str = every_sent.split('\n')
        for every_str in lst_str:
            if 'NOUN' in every_str:
                order, token, head, rel_type, pos = parsing_things(every_str)
                vp[num_sent].append(order)
        num_sent += 1
    return vp

In [ ]:
def find_adjs(parsed_text):
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    all_num_sent = count_sent(parsed_text)
    vp = {}
    for sent in range(1, all_num_sent+1):
        vp[sent] = []
    num_sent = 1
    for every_sent in sent_lst:
        lst_str = every_sent.split('\n')
        for every_str in lst_str:
            if 'ADJ' in every_str:
                order, token, head, rel_type, pos = parsing_things(every_str)
                vp[num_sent].append(head)
        num_sent += 1
    return vp

In [ ]:
def adj_noun(parsed_text):
    adjs = find_adjs(parsed_text)
    nouns = find_nouns(parsed_text)
    adj_noun = {}
    for key in adjs:
        adj_noun[key] = len([i for i, j in zip(adjs[key], nouns[key]) if i == j])
    return adj_noun

#### Gerunds and infinitives in subject position

In [ ]:
def find_ger_inf(parsed_text):
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    all_num_sent = count_sent(parsed_text)
    poss = {}
    for sent in range(1, all_num_sent+1):
        poss[sent] = 0
    num_sent = 1
    for every_sent in sent_lst:
        lst_str = every_sent.split('\n')
        for every_str in lst_str:
            if 'VerbForm=Ger' in every_str or 'VerbForm=Inf' in every_str and 'xcomp' not in every_str:
                order, token, head, rel_type, pos = parsing_things(every_str)
                poss[num_sent] = poss[num_sent] + 1
        num_sent += 1
    return poss

#### Nouns plus participle

In [ ]:
def find_parts(parsed_text):
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    all_num_sent = count_sent(parsed_text)
    vp = {}
    for sent in range(1, all_num_sent+1):
        vp[sent] = []
    num_sent = 1
    for every_sent in sent_lst:
        lst_str = every_sent.split('\n')
        for every_str in lst_str:
            if 'VerbForm=Part' in every_str:
                order, token, head, rel_type, pos = parsing_things(every_str)
                vp[num_sent].append(head)
        num_sent += 1
    return vp

In [ ]:
def parts_noun(parsed_text):
    parts = find_parts(parsed_text)
    nouns = find_nouns(parsed_text)
    adj_noun = {}
    for key in parts:
        adj_noun[key] = len([i for i, j in zip(parts[key], nouns[key]) if i == j])
    return adj_noun

In [ ]:
def count(d):
    num = 0
    for key, value in d.items():
        num = num + value
    return num

### 10. Количество глагольных групп

In [ ]:
def find_verbs(parsed_text):
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    all_num_sent = count_sent(parsed_text)
    vp = {}
    for sent in range(1, all_num_sent+1):
        vp[sent] = []
    num_sent = 1
    for every_sent in sent_lst:
        lst_str = every_sent.split('\n')
        for every_str in lst_str:
            if 'VERB' in every_str or 'AUX' in every_str:
                order, token, head, rel_type, pos = parsing_things(every_str)
                vp[num_sent].append(order)
        num_sent += 1
    return vp

In [ ]:
def find_vp(parsed_text):
    vp = find_verbs(parsed_text)
    sent_lst = re.findall('(1\t.+?)\n\n', parsed_text, re.DOTALL)
    all_num_sent = count_sent(parsed_text)
    num_sent = 1
    vp_final = {}
    for sent in range(1, all_num_sent+1):
        vp_final[sent] = 0
    for every_sent in sent_lst:
        #print('НОМЕР ПРЕДЛОЖЕНИЯ: ' + str(num_sent))
        for every_vp in vp[num_sent]:
            #print(every_vp)
            finding_dep = re.findall('\t'+every_vp+'\t.+?\t', every_sent)
            finding_dep_2 = []
            for every_finding_dep in finding_dep:
                find_del = re.search('mark|nsubj|punct', every_finding_dep)
                if find_del is None:
                    finding_dep_2.append(every_finding_dep)
            if finding_dep_2 != []:
                vp_final[num_sent] += 1 
        num_sent += 1
    return vp_final

In [ ]:
def count_vp(parsed_text):
    vp_final = find_vp(parsed_text)
    num_vp = 0
    for key, value in vp_final.items():
        num_vp = num_vp + value
    return num_vp

In [ ]:
#find_verbs(parsed_text)

In [ ]:
#find_vp(parsed_text)

In [ ]:
#count_vp(parsed_text)

In [ ]:
#f = open('test2.tsv', 'w')
#f.write(parsed_text)
#f.close()

In [ ]:
#parsed_text = get_parsed_text('english-partut-ud-2.0-170801.udpipe', '/Users/irene/Desktop/Курсовая/rr.txt')